# Create Heatmap for EGFR, PIK3CA, and PTEN

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

In [46]:
df = pd.read_csv('csv/all_heatmap.csv') # use local files (with cis)

In [47]:
genes = ['PTEN','EGFR','PIK3CA', 'KRAS', 'HRAS','NRAS', 'TP53','AKT','ATM', 'RB1']
bool_df = df.Proteomics.isin(genes)
plot_df = df[bool_df]
len(plot_df.Proteomics.unique())


9

In [48]:
# Only include p-values < a 
a = 0.1
plot_df = plot_df.loc[plot_df['P_Value'] <= a]
#plot_df.loc[plot_df['Proteomics'] == 'PTEN']
plot_df

,Proteomics,P_Value,Medians,Cancer
12,PTEN,0.000100,-0.511726,GBM
11194,PTEN,0.004396,-0.318812,HNSCC
11584,PIK3CA,0.039105,0.163663,HNSCC
22887,EGFR,0.003209,2.329700,LUAD
33584,PTEN,0.000234,-0.820150,LSCC
33893,PIK3CA,0.099452,0.272600,LSCC
45159,PTEN,0.043660,-1.008000,BR
66209,PTEN,0.025287,-0.746600,EC
66286,TP53,0.037681,-0.802600,EC
66341,PIK3CA,0.045727,-0.353000,EC


In [49]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 300, legend_min = 1e-3, legend_med = 1e-2, legend_max = 5e-2)

Loading BokehJS ...

Figure(id='4007', ...)

In [13]:
plot_df.loc[plot_df['Proteomics'] == 'EGFR']

,Proteomics,P_Value,Medians,Cancer,size2,size
22885,EGFR,0.003209,2.3297,LUAD,5.741755,17.225266


# Check luad has actual data for EGFR

In [19]:
l = cptac.Luad()

cptac warning: The LUAD dataset is under publication embargo until July 01, 2020. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 1)


In [29]:
# Returns a dataframe with proteomics and mutation type

def format_df(cancer_object, trans, gene_in = 'PTEN', utils = u):
    mut_type = cancer_object.get_genotype_all_vars(gene_in)
    
    if cancer_object.get_cancer_type() not in ('luad'):
        # Keep only tumor samples from proteomics
        prot_and_mutations = cancer_object.join_omics_to_mutations(
            mutations_genes = [gene_in], omics_df_name = 'proteomics', omics_genes = trans)
        prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
        
        # Reduce a multiindex 
        if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
            prot_and_mutations = utils.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)

        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        mut_type = mut_type[['Mutation']] 
        prot_df = prot_and_mutations.iloc[:,:-4] # Keep only proteomics
        merged = prot_df.join(mut_type)
        
        # Keep only Wildtype and deletion
        compare = ['Wildtype_Tumor','Deletion']
        get = merged['Mutation'].isin(compare)
        del_wt = merged[get]
    
    
    # Luad has no somatic mutations for PTEN which changes some things
    else: 
        # get_genotype_all_vars add cnv data under the column PTEN
        mut_type = mut_type.drop(columns=gene_in)
        # different code because no somatic mutation data for pten (can't join to somatic mutations)
        omics = cancer_object.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics',genes1= gene_in, 
            genes2=trans)
        omics = utils.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
        omics = omics.drop(columns='PTEN_CNV')
        # Get only tumor samples
        p = cancer_object.get_proteomics(tissue_type='tumor')
        tumor_ids = list(p.index)
        get = omics.index.isin(tumor_ids)
        omics = omics[get]
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        merged = omics.join(mut_type) 
        # Keep only No_Mutation (wildtype) and deletion
        compare = ['No_Mutation','Deletion']
        get = merged['Mutation'].isin(compare)
        del_wt = merged[get]
        del_wt['Mutation'] = np.where(
            del_wt['Mutation'] == 'No_Mutation', 'Wildtype_Tumor', 'Deletion')

    return del_wt


In [34]:
test = format_df(l, 'EGFR')
test['EGFR_proteomics'].value_counts()

cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-00001.N, C3L-00009.N, C3L-00080.N, C3L-00083.N, C3L-00093.N, C3L-00094.N, C3L-00095.N, C3L-00140.N, C3L-00144.N, C3L-00263.N, C3L-00279.N, C3L-00368.N, C3L-00412.N, C3L-00422.N, C3L-00510.N, C3L-00604.N, C3L-00893.N, C3L-00913.N, C3L-00973.N, C3L-01330.N, C3L-01632.N, C3L-01682.N, C3L-01683.N, C3L-01889.N, C3L-01890, C3L-01890.N, C3L-01924.N, C3L-02219.N, C3L-02345.N, C3L-02348.N, C3L-02350.N, C3L-02365.N, C3L-02508.N, C3L-02549.N, C3N-00167.N, C3N-00169.N, C3N-00175.N, C3N-00180.N, C3N-00199.N, C3N-00203.N, C3N-00217.N, C3N-00223.N, C3N-00293.N, C3N-00433.N, C3N-00546.N, C3N-00547.N, C3N-00549.N, C3N-00550.N, C3N-00551.N, C3N-00552.N, C3N-00556.N, C3N-00559.N, C3N-00560.N, C3N-00572.N, C3N-00574.N, C3N-00578.N, C3N-00579.N, C3N-00580.N, C3N-00704.N, C3N-00737.N, C3N-00738.N, C3N-00959.N, C3N-01016.N, C3N-01021.N, C3N-01023.N, C3N-01024.N, C3N-01030.N, C3N-01

 1.0362    1
-0.1776    1
-0.5937    1
-0.3358    1
-0.7235    1
          ..
 1.6805    1
 2.3528    1
 0.3395    1
-2.8313    1
-0.3750    1
Name: EGFR_proteomics, Length: 105, dtype: int64